## Quick check any physics parameters for a single file

In [9]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, sys, pathlib, shlex, subprocess
import xbout
import scipy
import xhermes
from xhermes import *


sys.path.append(r"/users/jpm590/2dspace/post-processing/sdtools/")


from hermes3.utils import *


from hermes3.fluxes import *
from hermes3.case_db import *
from hermes3.load import *
from hermes3.named_selections import *
from hermes3.plotting import *
from hermes3.grid_fields import *
from hermes3.accessors import *
from hermes3.selectors import *

%load_ext autoreload
%autoreload 2
print("Done")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Done


In [10]:
db = CaseDB(
    case_dir = r"/users/jpm590/scratch/",
    # grid_dir = r"/users/jpm590/2dspace/hermes-3/build-mc-master"
    grid_dir = r"/users/jpm590/neutralrun/hermes-3/master"

)

toload = [
    dict(name="source_guess", id="260112-cdn-46895-david-param", unnormalise_geom = True, use_xhermes = True, squash = True),
    dict(name="david_param", id="260105-cdn-46895-david-param", unnormalise_geom = True, use_xhermes = True, squash = True),


    # dict(name="new_grid", id="251210-cdn-46895", unnormalise_geom = True, use_xhermes = True, squash = True),
    dict(name="original", id="251007-2D-MASTU", unnormalise_geom = True, use_xhermes = True, squash = True)
    

]
cs = {}
for case in toload:
    cs[case["name"]] = db.load_case_2D(case["id"], use_squash = case["squash"], verbose = True)
    cs[case["name"]].extract_2d_tokamak_geometry()

- Reading case 260112-cdn-46895-david-param
-----------------------
- Looking for squash file
- Squash file found. squash date 01/12/2026, 08:56:13, dmp file date 01/12/2026, 09:51:44
- dmp files are newer than the squash file! Recreating...


- Done


/users/jpm590/workspace/hermes-3/venv/lib/python3.12/site-packages/xbout/load.py:1124: UserWarning: Will drop all variables containing the dimensions 'closed_wall' because they are not recognised
  warn(



Unnormalising with xHermes
- Reading case 260105-cdn-46895-david-param
-----------------------
- Looking for squash file
- Squash file found. squash date 01/09/2026, 07:47:48, dmp file date 01/08/2026, 16:26:36


/users/jpm590/workspace/hermes-3/venv/lib/python3.12/site-packages/xbout/load.py:1124: UserWarning: Will drop all variables containing the dimensions 'closed_wall' because they are not recognised
  warn(



Unnormalising with xHermes
- Reading case 251007-2D-MASTU
-----------------------
- Looking for squash file
- Squash file found. squash date 10/17/2025, 15:41:40, dmp file date 10/16/2025, 00:59:55

Unnormalising with xHermes


In [7]:
### Check meta data
m = cs["original"].ds.metadata
# m
ds = cs["original"].ds
# ds



### Calculate the total power source
By `Pd+_src` + `Pe_src` times dv

Energy = 3/2 * pressure * volume 

`P_src` are in per second. So it's Power = 3/2 * P_src * dv

In [ ]:
######
ds = cs["david_param"].ds.isel(t=-1)
total_press = ds["Pd+_src"].values + ds["Pe_src"].values + ds["Pd_src"].values
power = 3/2 * (ds["Pd+_src"].values + ds["Pe_src"].values + ds["Pd_src"].values) * ds["dv"].values
power = np.sum(power)
print(f"Total power source = {power}")



[[      0.              0.              0.        ...       0.
        0.              0.       ]
 [      0.              0.              0.        ...       0.
        0.              0.       ]
 [2245500.5782164 2245500.5782164       0.        ...       0.
  2245500.5782164 2245500.5782164]
 ...
 [      0.              0.              0.        ...       0.
        0.              0.       ]
 [      0.              0.              0.        ...       0.
        0.              0.       ]
 [      0.              0.              0.        ...       0.
        0.              0.       ]]
Total power source = 2877883.042521006


In [11]:

######
ds = cs["source_guess"].ds.isel(t=-1)
total_press = ds["Pd+_src"].values + ds["Pe_src"].values + ds["Pd_src"].values
print(total_press.sum())
power2 = 3/2 * (ds["Pd+_src"].values + ds["Pe_src"].values + ds["Pd_src"].values) * ds["dv"].values
power2 = np.sum(power2)
print(f"Total power source = {power2}")





198886430.26838717
Total power source = 1006911.3897414899


In [5]:
#######

ds = cs["original"].ds.isel(t=-1)

nparticles = (ds["Sd_src"].values) * ds["dv"].values
nparticles = np.sum(nparticles)
print(f"Total number of particles = {nparticles}")




#####
ds = cs["source_guess"].ds.isel(t=-1)
nparticles = (ds["Sd_src"].values) * ds["dv"].values
nparticles = np.sum(nparticles)
print(f"Total number of particles = {nparticles}")

#####

Total number of particles = 5e+21
Total number of particles = 1.3e+22


## Calculate volume

Get the volume of the region you want.

You could possibly choose ([reference](https://github.com/mikekryjak/sdtools/blob/4242b6f0a55edf66d0b6e6f706bb7dc8515eb143/hermes3/deprecated/stuff.py#L155)): 
```python 
    core = ds.hermesm.select_region("core_edge")
    sol = ds.hermesm.select_region("sol_edge")
    pfr = ds.hermesm.select_region("pfr_edge")
    domain = ds.hermesm.select_region("all_noguards")
    domain_volume = domain["dv"].values.sum()
```
When you change a grid, the core volume is changed as well, therefore check "core_edge"

In [6]:
ds = cs["original"].ds.isel(t=-1)

domain = ds.hermesm.select_region("core_edge")
domain_volume = domain["dv"].values.sum()
print(domain_volume)

#####
ds = cs["david_param"].ds.isel(t=-1)

domain = ds.hermesm.select_region("core_edge")
domain_volume = domain["dv"].values.sum()
print(domain_volume)

0.17736872858937297
0.2968858850770253


### Find the first core ring
The fuel is fuelling in the first core ring, so you should find the correct normalised position for fuelling function in input file.

```python
source = H(x) * H({first core ring} - x) * pressure_source
```

Below `sep` shows the position of the separatrix. `Srad` is the real distance from the separatrix (-: core, +: Sol). Note that `Srad` is in the cell center, while the real separatrix is on the boundary.


To get the normalised core ring position `x`, take `x` by cumulatively summing up the dx (width of each cell). Then normalised it by dividing max value.

The value you should take is the `x` where `Pe_src` is non-zero (1st core ring)

In [ ]:

ds = cs["david_param"].ds.isel(t=-1)
df = get_1d_radial_data(ds, params = ["dx", "Pd_src","Pe_src","Pd+_src", "Sd_src", "Nd_src"], region = "omp")

df["x"] = df["dx"].cumsum()
# x is in the center of the cell
df["x"] = df["x"]/df["x"].max()
df
# use the first 0.08

Parameter Nd_src not found


,t,total_poloidal_distance,zeta,dr,dx,Pd_src,Pe_src,Pd+_src,Sd_src,Srad,sep,x
0,0.00157,3.831418,3.141593,0.007483,0.002440,0.000000,1.122750e+06,1.122750e+06,0.000000e+00,-0.058898,0,0.071421
1,0.00157,3.943389,3.141593,0.006791,0.002215,0.000000,1.122750e+06,1.122750e+06,0.000000e+00,-0.051761,0,0.136254
2,0.00157,4.049806,3.141593,0.006145,0.002003,0.000000,1.122750e+06,1.122750e+06,0.000000e+00,-0.045293,0,0.194888
3,0.00157,4.150752,3.141593,0.005542,0.001805,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-0.039449,0,0.247709
4,0.00157,4.246313,3.141593,0.004980,0.001619,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-0.034188,0,0.295104
5,0.00157,4.336584,3.141593,0.004459,0.001447,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-0.029469,0,0.337462
6,0.00157,4.421685,3.141593,0.003977,0.001288,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-0.025251,0,0.375170
7,0.00157,4.501782,3.141593,0.003534,0.001143,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-0.021496,0,0.408616
8,0.00157,4.577099,3.141593,0.003131,0.001010,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-0.018164,0,0.438186
9,0.00157,4.647935,3.141593,0.002767,0.000891,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-0.015215,0,0.464269


In [12]:

ds = cs["source_guess"].ds.isel(t=-1)
df = get_1d_radial_data(ds, params = ["dx", "Pd_src","Pe_src","Pd+_src", "Sd_src", "Nd_src"], region = "omp")

df["x"] = df["dx"].cumsum()
# x is in the center of the cell
df["x"] = df["x"]/df["x"].max()
df
# use the first 0.08

Parameter Nd_src not found


,t,total_poloidal_distance,zeta,dr,dx,Pd_src,Pe_src,Pd+_src,Sd_src,Srad,sep,x
0,0.00178,3.831418,3.141593,0.007483,0.002440,0.000000,1.122750e+06,1.122750e+06,0.000000e+00,-0.058898,0,0.071421
1,0.00178,3.943389,3.141593,0.006791,0.002215,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-0.051761,0,0.136254
2,0.00178,4.049806,3.141593,0.006145,0.002003,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-0.045293,0,0.194888
3,0.00178,4.150752,3.141593,0.005542,0.001805,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-0.039449,0,0.247709
4,0.00178,4.246313,3.141593,0.004980,0.001619,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-0.034188,0,0.295104
5,0.00178,4.336584,3.141593,0.004459,0.001447,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-0.029469,0,0.337462
6,0.00178,4.421685,3.141593,0.003977,0.001288,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-0.025251,0,0.375170
7,0.00178,4.501782,3.141593,0.003534,0.001143,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-0.021496,0,0.408616
8,0.00178,4.577099,3.141593,0.003131,0.001010,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-0.018164,0,0.438186
9,0.00178,4.647935,3.141593,0.002767,0.000891,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-0.015215,0,0.464269
